In [1]:
import numpy as np
import cvxpy as cp
import pickle

from problems.data_generation_SC import generate_problem_data
from problems.problem_definition_SC import BilevelProblem
from algorithms.barrier_blo import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD
from algorithms.BiC_GAFFA import BiC_GAFFA
from algorithms.BSG import BSG

n = 60

max_elapsed_time = 120

hparams = {
    'barrier_blo': {
        'M': 0.001,
        't': 0.01,
        'alpha_x': 0.00002,
        'alpha_y': 0.5,
        'beta_y': 1,
        'epsilon_x': 0.1,
        'epsilon_y': 0.001,
        'inner_max_iters': 1000000,
        'outer_max_iters': 100000
    },
    'blocc': {
        'gamma': 10,
        'alpha_x': 0.00002,
        'alpha_g_y': 0.001,
        'alpha_F_y': 0.001,
        'beta_g_y': 0.5,
        'beta_F_y': 0.5,
        'epsilon_x': 0.1,
        'epsilon_inner_y_g': 0.05,
        'epsilon_outer_y_g': 0.05,
        'epsilon_inner_y_F': 0.05,
        'epsilon_outer_y_F': 0.05,
        'maxmin_g_outer_max_iters': 5000000,
        'maxmin_F_outer_max_iters': 5000000,
        'maxmin_g_inner_max_iters': 5000000,
        'maxmin_F_inner_max_iters': 5000000,
        'main_max_iters': 1000
    },
    # 'bic_gaffa':{
    #     'alpha':0.000001,
    #     'c':1,
    #     'tau':1.1,
    #     'gamma_1':10,
    #     'gamma_2':1,
    #     'eta':1,
    #     'r':1,
    #     'epsilon':0.1,
    #     'max_iters':300
    # },
    'BSG':{
        'alpha_x':0.000002,
        'alpha_y':0.001,
        'alpha_z':0.02,
        'epsilon_x':0.1,
        'epsilon_y':0.01,
        'epsilon_z':0.01,
        'max_iters_x':1000000,
        'max_iters_y':1000000,
        'max_iters_z':1000000
    },
    # 'IGD': {
    #     'M': 1e-3,
    #     't': 1e-3,
    #     'alpha_x': 0.001,
    #     'alpha_y': 0.001,
    #     'epsilon_x': 1e-3,
    #     'epsilon_y': 1e-3,
    #     'inner_max_iters': 1000,
    #     'outer_max_iters': 1000
    # }
}

seed = 8
data = generate_problem_data(n, seed)
problem = BilevelProblem(data)
x_init = np.zeros(n)
y_init = np.zeros(n)
z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
theta_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)

In [ ]:
for seed in range(10):
    print(f"------------testing seed {seed}------------")
    data = generate_problem_data(n, seed)
    problem = BilevelProblem(data)
    x_init = np.zeros(n)
    y_init = np.zeros(n)
    z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    theta_init = np.zeros(n)
    y_g_init = np.zeros(n)
    y_F_init = np.zeros(n)
    mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    
    # BSG
    bsg_algo = BSG(problem, hparams)
    x_opt_bsg, y_opt_bsg, history_bsg = bsg_algo.bsg(x_init, y_init, z_init, max_elapsed_time)
    f_bsg_num = problem.f(x_opt_bsg, y_opt_bsg)
    print(f_bsg_num)
    
    barrier_algo = BarrierBLO(problem, hparams)
    y_original_opt_bsg = barrier_algo.Interior_inner_loop(x_opt_bsg, y_opt_bsg)
    f_bsg_theo = problem.f(x_opt_bsg, y_original_opt_bsg)
    g_bsg_theo = problem.g(x_opt_bsg, y_original_opt_bsg)
    g_bsg_num = problem.g(x_opt_bsg, y_opt_bsg)

    h1_values = [problem.h_1(x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h2)]
    h_total_bsg = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_bsg_theo}, f_bsg_numerical_result={f_bsg_num}")
    print(f"g_theoretical_opt_result={g_bsg_theo}, g_bsg_numerical_result={g_bsg_num}")
    print(h_total_bsg)
    
    # BLOCC
    blocc_algo = BLOCC(problem, hparams)
    x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(x_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time)
    f_blocc_num = problem.f(x_opt_blocc, y_opt_blocc)
    print(f_blocc_num)
    
    barrier_algo = BarrierBLO(problem, hparams)
    y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
    f_blocc_theo = problem.f(x_opt_blocc, y_original_opt_blocc)
    g_blocc_theo = problem.g(x_opt_blocc, y_original_opt_blocc)
    g_blocc_num = problem.g(x_opt_blocc, y_opt_blocc)

    h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]
    h_total_blocc = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_blocc_theo}, f_blocc_numerical_result={f_blocc_num}")
    print(f"g_theoretical_opt_result={g_blocc_theo}, g_blocc_numerical_result={g_blocc_num}")
    print(h_total_blocc)
    
    # BFBM
    barrier_algo = BarrierBLO(problem, hparams)
    x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(x_init, y_init, max_elapsed_time)
    f_barrier_num = problem.f(x_opt_barrier, y_opt_barrier)
    print(f_barrier_num)
    
    y_original_opt_barrier = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
    f_barrier_theo = problem.f(x_opt_barrier, y_original_opt_barrier)
    g_barrier_theo = problem.g(x_opt_barrier, y_original_opt_barrier)
    g_barrier_num = problem.g(x_opt_barrier, y_opt_barrier)

    h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
    h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_barrier_theo}, f_barrier_numerical_result={f_barrier_num}")
    print(f"g_theoretical_opt_result={g_barrier_theo}, g_barrier_numerical_result={g_barrier_num}")
    print(h_total_barrier)
    
    # save all data
    results = {
        'seed': seed,
        'BSG': {
            'x_opt': x_opt_bsg,
            'y_opt': y_opt_bsg,
            'history': history_bsg,
            'y_original_opt': y_original_opt_bsg,
            'f_theoretical_opt': f_bsg_theo,
            'f_numerical': f_bsg_num,
            'g_theoretical_opt': g_bsg_theo,
            'g_numerical': g_bsg_num,
            'h_total': h_total_bsg
        },
        'blocc': {
            'x_opt': x_opt_blocc,
            'y_opt': y_opt_blocc,
            'history': history_blocc,
            'y_original_opt': y_original_opt_blocc,
            'f_theoretical_opt': f_blocc_theo,
            'f_numerical': f_blocc_num,
            'g_theoretical_opt': g_blocc_theo,
            'g_numerical': g_blocc_num,
            'h_total': h_total_blocc
        },
        'barrier': {
            'x_opt': x_opt_barrier,
            'y_opt': y_opt_barrier,
            'history': history_barrier,
            'y_original_opt': y_original_opt_barrier,
            'f_theoretical_opt': f_barrier_theo,
            'f_numerical': f_barrier_num,
            'g_theoretical_opt': g_barrier_theo,
            'g_numerical': g_barrier_num,
            'h_total': h_total_barrier
        }
    }

    save_filename = f'results_SC_seed_{seed}.pkl'

    with open(save_filename, 'wb') as f:
        pickle.dump(results, f)

    print(f"seed {seed} Results saved to {save_filename}")

In [54]:
hparams = {
    'barrier_blo': {
        'M': 0.001,
        't': 0.01,
        'alpha_x': 0.0005,
        'alpha_y': 0.5,
        'beta_y': 1,
        'epsilon_x': 0.1,
        'epsilon_y': 0.001,
        'inner_max_iters': 1000000,
        'outer_max_iters': 100000
    },
    'blocc': {
        'gamma': 10,
        'alpha_x': 0.001,
        'alpha_g_y': 0.001,
        'alpha_F_y': 0.001,
        'beta_g_y': 0.5,
        'beta_F_y': 0.5,
        'epsilon_x': 0.1,
        'epsilon_inner_y_g': 0.05,
        'epsilon_outer_y_g': 0.05,
        'epsilon_inner_y_F': 0.05,
        'epsilon_outer_y_F': 0.05,
        'maxmin_g_outer_max_iters': 5000000,
        'maxmin_F_outer_max_iters': 5000000,
        'maxmin_g_inner_max_iters': 5000000,
        'maxmin_F_inner_max_iters': 5000000,
        'main_max_iters': 1000
    },
    # 'bic_gaffa':{
    #     'alpha':0.000001,
    #     'c':1,
    #     'tau':1.1,
    #     'gamma_1':10,
    #     'gamma_2':1,
    #     'eta':1,
    #     'r':1,
    #     'epsilon':0.1,
    #     'max_iters':300
    # },
    'BSG':{
        'alpha_x':0.00005,
        'alpha_y':0.001,
        'alpha_z':0.02,
        'epsilon_x':0.1,
        'epsilon_y':0.01,
        'epsilon_z':0.01,
        'max_iters_x':1000000,
        'max_iters_y':1000000,
        'max_iters_z':1000000
    },
    # 'IGD': {
    #     'M': 1e-3,
    #     't': 1e-3,
    #     'alpha_x': 0.001,
    #     'alpha_y': 0.001,
    #     'epsilon_x': 1e-3,
    #     'epsilon_y': 1e-3,
    #     'inner_max_iters': 1000,
    #     'outer_max_iters': 1000
    # }
}

In [ ]:
# diminish tests
for seed in range(10):
    print(f"------------testing seed {seed}------------")
    data = generate_problem_data(n, seed)
    problem = BilevelProblem(data)
    x_init = np.zeros(n)
    y_init = np.zeros(n)
    z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    theta_init = np.zeros(n)
    y_g_init = np.zeros(n)
    y_F_init = np.zeros(n)
    mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    
    # BSG
    bsg_algo = BSG(problem, hparams)
    x_opt_bsg, y_opt_bsg, history_bsg = bsg_algo.bsg(x_init, y_init, z_init, max_elapsed_time, step_size_type="diminish")
    f_bsg_num = problem.f(x_opt_bsg, y_opt_bsg)
    print(f_bsg_num)
    
    barrier_algo = BarrierBLO(problem, hparams)
    y_original_opt_bsg = barrier_algo.Interior_inner_loop(x_opt_bsg, y_opt_bsg)
    f_bsg_theo = problem.f(x_opt_bsg, y_original_opt_bsg)
    g_bsg_theo = problem.g(x_opt_bsg, y_original_opt_bsg)
    g_bsg_num = problem.g(x_opt_bsg, y_opt_bsg)

    h1_values = [problem.h_1(x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h2)]
    h_total_bsg = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_bsg_theo}, f_bsg_numerical_result={f_bsg_num}")
    print(f"g_theoretical_opt_result={g_bsg_theo}, g_bsg_numerical_result={g_bsg_num}")
    print(h_total_bsg)
    
    # BLOCC
    blocc_algo = BLOCC(problem, hparams)
    x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(x_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time, step_size_type="diminish")
    f_blocc_num = problem.f(x_opt_blocc, y_opt_blocc)
    print(f_blocc_num)
    
    barrier_algo = BarrierBLO(problem, hparams)
    y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
    f_blocc_theo = problem.f(x_opt_blocc, y_original_opt_blocc)
    g_blocc_theo = problem.g(x_opt_blocc, y_original_opt_blocc)
    g_blocc_num = problem.g(x_opt_blocc, y_opt_blocc)

    h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]
    h_total_blocc = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_blocc_theo}, f_blocc_numerical_result={f_blocc_num}")
    print(f"g_theoretical_opt_result={g_blocc_theo}, g_blocc_numerical_result={g_blocc_num}")
    print(h_total_blocc)
    
    # BFBM
    barrier_algo = BarrierBLO(problem, hparams)
    x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(x_init, y_init, max_elapsed_time, step_size_type="diminish")
    f_barrier_num = problem.f(x_opt_barrier, y_opt_barrier)
    print(f_barrier_num)
    
    y_original_opt_barrier = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
    f_barrier_theo = problem.f(x_opt_barrier, y_original_opt_barrier)
    g_barrier_theo = problem.g(x_opt_barrier, y_original_opt_barrier)
    g_barrier_num = problem.g(x_opt_barrier, y_opt_barrier)

    h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
    h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_barrier_theo}, f_barrier_numerical_result={f_barrier_num}")
    print(f"g_theoretical_opt_result={g_barrier_theo}, g_barrier_numerical_result={g_barrier_num}")
    print(h_total_barrier)
    
    # save all data
    results = {
        'seed': seed,
        'BSG': {
            'x_opt': x_opt_bsg,
            'y_opt': y_opt_bsg,
            'history': history_bsg,
            'y_original_opt': y_original_opt_bsg,
            'f_theoretical_opt': f_bsg_theo,
            'f_numerical': f_bsg_num,
            'g_theoretical_opt': g_bsg_theo,
            'g_numerical': g_bsg_num,
            'h_total': h_total_bsg
        },
        'blocc': {
            'x_opt': x_opt_blocc,
            'y_opt': y_opt_blocc,
            'history': history_blocc,
            'y_original_opt': y_original_opt_blocc,
            'f_theoretical_opt': f_blocc_theo,
            'f_numerical': f_blocc_num,
            'g_theoretical_opt': g_blocc_theo,
            'g_numerical': g_blocc_num,
            'h_total': h_total_blocc
        },
        'barrier': {
            'x_opt': x_opt_barrier,
            'y_opt': y_opt_barrier,
            'history': history_barrier,
            'y_original_opt': y_original_opt_barrier,
            'f_theoretical_opt': f_barrier_theo,
            'f_numerical': f_barrier_num,
            'g_theoretical_opt': g_barrier_theo,
            'g_numerical': g_barrier_num,
            'h_total': h_total_barrier
        }
    }

    save_filename = f'results_SC_diminish_seed_{seed}.pkl'

    with open(save_filename, 'wb') as f:
        pickle.dump(results, f)

    print(f"seed {seed} Results saved to {save_filename}")